In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import graphviz
from collections import namedtuple

## Load Abalone
`load_abalone` loads the csv file from the provided path and sets the column names. Also deletes unwanted features from the dataset.

In [2]:
def load_abalone(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["sex", "length", "diameter", "height", "whole weight", "shucked weight",
                 "viscera weight", "shell weight", "rings"], axis=1, inplace=True)
    df = df.drop(columns = ['sex'])
    return df

## Load Breast Cancer
`load_breast_cancer` loads the csv file from the provided path and sets the column names. Also deletes unwanted features from the dataset and imputes missing values with the mean of the column.

In [3]:
def load_breast_cancer(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["sample code number", "clump thickness", "uniformity of cell size", "uniformity of cell shape",
                 "marginal adhesion", "single epithelial cell size",
                 "bare nuclei", "bland chromatin", "normal nucleoli", "mitoses", "class"], axis=1, inplace=True)
    df = df.drop(columns = ["sample code number"])
    df = df.replace('?', np.nan)
    df['bare nuclei'] = df['bare nuclei'].astype(float)
    df = impute_mean(df, 'bare nuclei', int)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

## Load Car
`load_car` loads the csv file from the provided path and sets the column names.

In [4]:
def load_car(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"], axis=1, inplace=True)
    return df

## Load Forest Fires
`load_forest_fires` loads the csv file from the provided path and sets the column names. Also deletes unwanted features from the dataset.

In [5]:
def load_forest_fires(path):
    df = pd.read_csv(path)
    df = df.drop(columns=['X', 'Y', 'month', 'day'])
    return df

## Load House Votes
`load_house_votes` loads the csv file from the provided path and sets the column names.

In [6]:
def load_house_votes(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["class", "handicapped infants", "water project cost sharing", "adoption of the budget resolution",
                 "physician fee freeze", "el salvador aid", "religious groups in schools", "anti satellite test ban",
                 "aid to nicaraguan contras", "mx missile", "immigration", "synfuels corporation cutback",
                 "education spending", "superfund right to sue", "crime", "duty free exports",
                 "export administration act south africa"], axis=1, inplace=True)
    return df

## Load Machine
`load_machine` loads the csv file from the provided path and sets the column names. Also deletes unwanted features from the dataset.

In [7]:
def load_machine(path):
    df = pd.read_csv(path, header = None)
    df.set_axis(["vendor", "model", "MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"], axis=1, inplace=True)
    df = df.drop(["vendor","model", "ERP"], axis=1)
    return df

## Impute Mean
`impute_mean` updates the missing values in a column of the dataset with the mean of the specified column

In [8]:
def impute_mean(data, column, data_type):
    _data = deepcopy(data)
    mean = _data[column].mean()
    _data[column] = _data[column].fillna(mean)
    if data_type == int:
        _data[column] = _data[column].astype(np.int64)
    return _data

## Encode Ordinal
`encode_ordinal` encodes ordinal values of a feature based on the relationship list provided.

In [9]:
def encode_ordinal(data, column, relationship):
    _data = deepcopy(data)
    for index, r in enumerate(relationship):
        _data[column] = _data[column].replace(r, index)
        print(f"encoding column: {column} - value: {r} as: {index}")
    return _data

## One Hot Encode
`one_hot_encode` encodes features of the desired columns with one hot values.

In [10]:
def one_hot_encode(data, columns):
    _data = deepcopy(data)
    for _column in columns:
        _data2 = pd.get_dummies(_data[_column]).groupby(level=0,axis=1).max().add_prefix(_column + ' - ')
        _data = pd.concat([_data, _data2], axis=1)
        _data = _data.drop([_column], axis=1)
    return _data

## Discretize Feature
`discretize_feature` bins feature values based on the desired bin type. Frequency type tries to ensure the same number of data points are in each bin and bins can vary in width. Width type specifies the bin width so the number on points in each bin can vary

In [11]:
def discretize_feature(data, columns, number_bins, type_bins):
    _data = deepcopy(data)
    for _column in columns:
        if type_bins == "frequency":
            _data2 = pd.qcut(_data[_column], q = number_bins, precision=0, duplicates='drop')
            _data[_column] = _data2

        if type_bins == "width":
            _data2 = pd.cut(_data[_column], number_bins)
            _data[_column] = _data2

    return _data

## Standardize
`standardize` updates values in training data and test data to the z standard using the mean and standard deviation from the training set for both calculations

In [12]:
def standardize(training_data, test_data, columns):
    _data_train = deepcopy(training_data)
    _data_test = deepcopy(test_data)
    for _column in columns:
        mean = _data_train[_column].mean()
        std = _data_train[_column].std()
        z_train = (_data_train[_column] - mean)/std
        z_test = (_data_test[_column] - mean)/std

        _data_train[_column] = z_train
        _data_test[_column] = z_test
    return _data_train, _data_test

## Extract Validation Set
`extract_validation_set` extracts a random 20% sample for the validation set. Evenly samples the groupings of the output. Randomly shuffles the remaining 80%.

In [13]:
def extract_validation_set(data, class_column):
    _data = deepcopy(data)
    y = class_column

    stratify_20 = _data.groupby(y, group_keys=False).sample(frac=0.20)
    stratify_80 = _data.drop(stratify_20.index).sample(frac=1)

    return stratify_80, stratify_20

## Create Train Test
`create_train_test` takes in the folds of a k-fold to create a test set from 1 fold and a training set from the remaing 4 folds

In [14]:
def create_train_test(folds, index):
    training = pd.DataFrame()
    test = []
    for i, fold in enumerate(folds):
        if i == index:
            test = fold
        else:
            training = pd.concat([training, fold])
    return training, test

## Stratified K Fold
`stratified_k_fold` takes the provided dataset and creates a specified number of evenly distributed folds of that data while maintaining output grouping distributions

In [15]:
def stratified_k_fold(data, k, class_column):
    y = class_column
    split = []
    unique_keys = data.value_counts(subset=y, normalize=True).keys()
    split_class = [data.loc[data[y] == keys] for keys in unique_keys]

    for class_value in split_class:
        d, m = divmod(len(class_value), k)
        split.append(list(class_value[i * d + min(i, m):(i + 1) * d + min(i + 1, m)] for i in range(k)))

    folds = [pd.concat([split[i][c] for i in range(len(unique_keys))]) for c in range(k)]

    return folds

## K Fold
`k_fold` takes the provided dataset and creates a specified number of evenly distributed folds of that data

In [16]:
def k_fold(data, k):

    d, m = divmod(len(data), k)
    folds = list(data[i * d + min(i, m):(i + 1) * d + min(i + 1, m)] for i in range(k))

    return folds

## K Fold Cross Validation Sets
`k_fold_cross_validation_sets` splits the data into validation and train/test sets. Folds the sets k times. Can fold with stratification or without stratification

In [17]:
def k_fold_cross_validation_sets(data, k, class_column, stratified=True, validation=True):
    if validation:
        train, validation = extract_validation_set(data, class_column)
        if stratified:
            train = stratified_k_fold(train, k, class_column)
            #validation = stratified_k_fold(validation, k, class_column)
        else:
            train = k_fold(train, k)
            #validation = k_fold(validation, k)
        return train, validation
    else:
        if stratified:
            train = stratified_k_fold(data, k, class_column)
        else:
            train = k_fold(data, k)
    return train

## Evaluation
`evaluation` applies an evaluation metric on the predicted values. Able to use a classification score by comparing ground truth to the predicted values to determine how many are correct. Able to use Mean Square Error to determine the distance between the ground truth and the predicted values

In [18]:
def evaluation(ground_truth, predicted_values, metric):
    if metric == 'classification_score':
        count = 0
        for index, value in enumerate(ground_truth):
            if predicted_values[index] == value:
                count += 1
        count = count/len(ground_truth)
        return count
    if metric == 'mse':
        error = sum((np.array(ground_truth) - np.array(predicted_values))**2)/len(ground_truth)
        return error

## Prior Entropy
`prior_entropy` performs the entropy calculation of a dataset for each class in the dataset. Used for data in the form of a dataframe

In [19]:
def prior_entropy(data):
    #determines the number of each class in the dataset
    class_counts = data.value_counts()
    #calculate the entropy for the dataset
    entropy = -1 * sum((class_counts/len(data))*np.log2(class_counts/len(data)))
    return entropy

## Prior Entropy Numeric
`prior_entropy_numeric` performs the entropy calculation of a dataset for each class in the dataset. Used for data in the form of a numpy array

In [20]:
def prior_entropy_numeric(class_column):
    entropy = []
    #get the unique values of a class and the counts associated with each unique value
    unique, counts = np.unique(class_column, return_counts=True)
    data_length = len(class_column)
    #calculate the entropy for the dataset
    for value in counts:
        entropy.append(value/data_length * np.log2(value/data_length))
    entropy = -1 * sum(entropy)
    return entropy

## Expected Entropy Discrete
`expected_entropy_discrete` calculates the expected entropy after a split. The data is split by the feature value for the entropy calculation. Used for features with discrete values

In [21]:
def expected_entropy_discrete(data, feature, class_column):
    entropy = []
    intrinsic_value = []
    #calculate the unique feature values and their counts
    feature_value_counts = data[feature].value_counts()
    for key, value in feature_value_counts.items():
        #get a subset of the data by feature value
        subset = data[data[feature] == key]
        #calculate intrinsic value of the subset
        intrinsic_value.append((value/len(data)) * np.log2(value/len(data)))
        #calculate the entropy of the subset
        h = prior_entropy(subset[class_column])
        entropy.append(value/len(data) * h)
    intrinsic_value = -1 * sum(intrinsic_value)
    #total expected entropy for the feature
    entropy = sum(entropy)
    return entropy, intrinsic_value

## Expected Entropy Numeric
`expected_entropy_numeric` calculates the expected entropy after a split. Each sample were the value of a feature changes is considered for the split. The split value with the lowest entropy is selected as the split location for that feautre. Used for features with continuous numeric values

In [22]:
def expected_entropy_numeric(data, feature_np, class_column):
    entropy, intrinsic_value, split_value = ([] for i in range(3))
    for index in range(len(data)-1):
        #split the data at the next sample
        split = (data[index][feature_np] + data[index+1][feature_np]) / 2
        subset1, subset2 = data[:index+1], data[index+1:]
        #if the two adjacent samples have the same value go to use the next sample as the split
        if data[index][feature_np] == data[index+1][feature_np]:
            continue
        #calculate intrinsic value of the split
        intrinsic_value.append(((len(subset1)/len(data)) * np.log2(len(subset1)/len(data))) + ((len(subset2)/len(data)) * np.log2(len(subset2)/len(data))))
        #calculate the entropy of the split
        h1, h2 = prior_entropy_numeric(np.transpose(subset1)[class_column]), prior_entropy_numeric(np.transpose(subset2)[class_column])
        entropy.append((len(subset1)/len(data) * h1) + (len(subset2)/len(data) * h2))
        split_value.append(split)
    if len(intrinsic_value) == 0:
        intrinsic_value.append(0)
        entropy.append(prior_entropy_numeric(np.transpose(data)[class_column]))
        split_value.append(np.inf)
    # find the split with the minimum entropy to use as the split value
    intrinsic_value = -1 * intrinsic_value[np.argmin(entropy)]
    split_value = split_value[np.argmin(entropy)]
    entropy = min(entropy)
    return entropy, intrinsic_value, split_value

## Gain Ratio
`gain_ratio` calculate the gain ratio for a feature

In [23]:
def gain_ratio(data, feature, class_column):
    prior = prior_entropy(data[class_column])
    #use numeric entropy calculation if feature values are type float
    if data[feature].dtypes == np.float64:
        class_column = data.columns.get_loc(class_column)
        feature_np = data.columns.get_loc(feature)
        data = data.sort_values([feature]).to_numpy()
        expected, intrinsic_value, split_value = expected_entropy_numeric(data, feature_np, class_column)
    #use discrete entropy calculation if features are not type float
    else:
        expected, intrinsic_value = expected_entropy_discrete(data, feature, class_column)
        split_value = None
    #calcualte the gain
    gain = round(prior, 5) - round(expected, 5)
    #calculate the gain ratio if a gain value is returned. Else return 0 gain ratio
    if gain != 0:
        gain_r = round(gain/intrinsic_value, 5)
    else:
        gain_r = 0
    return gain_r, split_value

## Best Feature Classification
`best_feature_classification` determines the best feature to split on by looking at the gain ratios produced by each feature. Used for classification decision tree.

In [24]:
def best_feature_classification(data, class_column):
    gains = {}
    best = [-1, -1]
    for key in data:
        #calculate a gain ratio if column is not the class
        if key != class_column:
            gains[key] = gain_ratio(data, key, class_column)
    #find the feature with the highest gain ratio
    for key, value in gains.items():
        if value[0] > best[1]:
            best[1] = value[0]
            best[0] = key
    return {best[0]: gains[best[0]]}

## Decision Tree Classification
`decision_tree_classification` creates a decision tree for classification using gain ratio as the splitting criteria

In [25]:
def decision_tree_classification(_data, class_column):
    data = deepcopy(_data)
    #named tuple structure to be used as label for each node
    attr = namedtuple('attr', 'split name gain_ratio samples value value_index output')
    #find current node entropy. If 0 or no more features to split on create a leaf node
    prior = round(prior_entropy(data[class_column]), 5)
    if (prior == 0) or (len(data.columns.tolist()) == 1):
        node = {'label': attr(split = None, name = None, gain_ratio = prior, samples = len(data), value = data[class_column].value_counts().sort_index().tolist(),
                              value_index= data[class_column].value_counts().sort_index().index.tolist(), output = data[class_column].value_counts().sort_index().idxmax())}
        return node
    #find best feature to split on
    best_f = best_feature_classification(data, class_column)
    #enter branch if feature is discrete
    if best_f[list(best_f)[0]][1] is None:
        #create a dictionary entry for the feature as a new node
        node = {list(best_f)[0]: {}}
        #create branches of the node based on best feature
        for branch in data[list(best_f)[0]].sort_values().unique():
            node[list(best_f)[0]][branch] = {}
        #add label to the node to store relevant information about the node
        node[list(best_f)[0]]['label'] = attr(split = best_f[list(best_f)[0]][1], name = str(list(best_f)[0]),
                                              gain_ratio = best_f[list(best_f)[0]][0], samples = len(data), value = data[class_column].value_counts().sort_index().tolist(),
                                              value_index= data[class_column].value_counts().sort_index().index.tolist(), output = data[class_column].value_counts().sort_index().idxmax())
        #continue creating the tree by going down each branch of the new node
        for key in node[list(best_f)[0]]:
            if key != 'label':
                #create subset of data that belongs to the branch
                subset = data[data[list(best_f)[0]] == key]
                subset = subset.drop(columns = [list(best_f)[0]])
                #if subset is empty load the class as the subset to have the next iteration create a leaf
                if subset.empty:
                    subset = data[class_column].to_frame()
                #find the next node or leaf
                node[list(best_f)[0]][key] = decision_tree_classification(subset, class_column)
        return node
    #enter branch if feature is numeric
    else:
        #create new dictionary entry for the feature as a new node and create the branches of the node
        node = {list(best_f)[0]: {'True': {}, 'False': {}}}
        #add label to the node to store relevant information about the node
        node[list(best_f)[0]]['label'] = attr(split = round(best_f[list(best_f)[0]][1], 5), name = str(list(best_f)[0]) + '<=' + str(round(best_f[list(best_f)[0]][1], 5)),
                                              gain_ratio = best_f[list(best_f)[0]][0], samples = len(data), value = data[class_column].value_counts().sort_index().tolist(),
                                              value_index= data[class_column].value_counts().sort_index().index.tolist(), output = data[class_column].value_counts().sort_index().idxmax())
        #split the data based on the split value return from finding the best feature
        subset1 = data[data[list(best_f)[0]] <= best_f[list(best_f)[0]][1]]
        subset2 = data[data[list(best_f)[0]] > best_f[list(best_f)[0]][1]]
        #find the next node or leaf if subset has samples
        if len(subset1) != 0:
            node[list(best_f)[0]]['True'] = decision_tree_classification(subset1, class_column)
        if len(subset2) != 0:
            node[list(best_f)[0]]['False'] = decision_tree_classification(subset2, class_column)
        return node

## DT Classify
`dt_classify` finds the output a tree predicts for the samples provided

In [26]:
def dt_classify(data, tree):
    results = []
    for index, sample in data.iterrows():
        results.append(traverse_tree(sample, tree))
    return results

## Traverse Tree
`traverse_tree` takes a sample and looks through the tree to find the predicted output

In [27]:
def traverse_tree(data, tree):
    #look at the node and determine what feature it represents
    feature = list(tree.keys())[0]
    #if label then a leaf was found. Return the leaf output value
    if feature == 'label':
        result = tree[feature].output
        return result
    #find the feature value from the sample
    value = data[feature]
    #if feature is numeric follow the if statement
    if list(tree[feature].keys())[0] == 'True':
        #find the split value of the node
        split = tree[feature]['label'].split
        #follow the appropriate split value down the tree
        if value <= split:
            result = traverse_tree(data, tree[feature]['True'])
        else:
            result = traverse_tree(data, tree[feature]['False'])
    #if feature is categorical follow the else statement
    else:
        #try to move down the tree. If feature value not in tree stop at this node to determine output value
        try:
            result = traverse_tree(data, tree[feature][value])
        except KeyError:
            result = tree[feature]['label'].output
    return result

## Find Leaves
`find_leaves` finds all the leaves of a tree and the path to the leaf

In [28]:
def find_leaves(tree):
    leaf = []
    for index, sub_tree in tree.items():
        if type(sub_tree) is dict:
            #if not leaf then keep going down the tree and save the path
            if list(sub_tree.keys())[0] != 'label':
                node = find_leaves(sub_tree)
                for key in node:
                    leaf.append([index, key])
            #add leaf to the result
            else:
                leaf.append([index, sub_tree])
    return leaf

## Flatten
`flatten` takes a nested list and flattens it into 1 list

In [29]:
def flatten(nested):
    flat = []
    for i in nested:
        if isinstance(i,list):
            flat.extend(flatten(i))
        else:
            flat.append(i)
    return flat

## Create Pruning Paths
`create_pruning_paths` creates a list of nodes for pruning including the path to the node

In [30]:
def create_pruning_paths(leaves):
    paths = []
    unique = []
    #take the list of leaves, flatten the lists, and delete path up to the node to be pruned
    for i in range(len(leaves)):
        paths.append(flatten(leaves[i]))
        del paths[i][-3:]
    #delete empty lists
    paths = [x for x in paths if x != []]
    #delete duplicate nodes
    for x in paths:
        if isinstance(x, list) and (x not in unique):
            unique.append(x)
    #sort by longest path to node
    unique.sort(reverse=True, key=len)
    return unique

## Find Pruning Nodes
`find_pruning_nodes` finds the nodes of a tree that can be pruned

In [31]:
def find_pruning_nodes(tree):
    leaves = find_leaves(tree)
    nodes = create_pruning_paths(leaves)
    return nodes

## Prune Nodes
`prune_nodes` deletes a node from the tree and replaces it with a leaf

In [32]:
def prune_node(tree, paths):
    if len(paths) == 0:
        x = tree[list(tree.keys())[0]]['label']
        tree.clear()
        tree['label'] = x
        return None
    return prune_node(tree[paths[0]], paths[1:])

## Num Paths
`num_paths` returns the number of paths in a tree and the length of the longest path

In [33]:
def num_paths(tree):
    leaves = find_leaves(tree)
    paths = []
    unique = []
    for i in range(len(leaves)):
        paths.append(flatten(leaves[i]))
    paths = [x for x in paths if x != []]
    for x in paths:
        if isinstance(x, list) and (x not in unique):
            unique.append(x)
    unique.sort(reverse=True, key=len)
    return len(unique), (len(unique[0])-1)/2

## Cross Validation Classification
`cross_validation_classification` K fold cross validation for a classification decision tree.

In [34]:
def cross_validation_classification(data, class_column, post_prune = True):
    eval_pre = []
    eval_post = []
    #creates the validation and test/train sets
    folds, validation = k_fold_cross_validation_sets(data, 5, class_column)
    #run each fold as the test set
    for index, fold in enumerate(folds):
        #create train test sets from folds
        train, test = create_train_test(folds, index)
        #create a decision tree
        tree = decision_tree_classification(train, class_column)
        #if pruning is selected then prune the tree
        if post_prune:
            #graph the un-pruned tree
            graph_tree(data, tree, data.name + ' pre', index)
            #determine performance of the un-pruned tree
            result = dt_classify(test, tree)
            eval_pre.append(evaluation(test[class_column], result, 'classification_score'))
            #determine number of paths in the tree
            path_count, length = num_paths(tree)
            new_path_count = 0
            #keep pruning until the number of paths in the tree stops changing
            while path_count > new_path_count:
                #find nodes to be pruned
                nodes = find_pruning_nodes(tree)
                #determine number of paths in the tree
                path_count, length = num_paths(tree)
                #test performance of pruning each node individually against the validation set
                for n in nodes:
                    #copy the tree
                    prune_tree = deepcopy(tree)
                    #determine performance of the tree
                    result = dt_classify(validation, prune_tree)
                    eval1 = evaluation(validation[class_column], result, 'classification_score')
                    #prune tree
                    prune_node(prune_tree, n)
                    #determine performance of pruned tree
                    result = dt_classify(validation, prune_tree)
                    eval2 = evaluation(validation[class_column], result, 'classification_score')
                    #if performance of pruned tree is not worse, then keep the node pruned
                    if eval2 >= eval1:
                        tree = prune_tree
                #count the new number of paths from the pruned tree
                new_path_count, length = num_paths(tree)
            #graph the pruned tree
            graph_tree(data, tree, data.name + ' post', index)
        #if no pruning then graph the tree
        else:
            graph_tree(data, tree, data.name, index)
        #evaluate performance of the tree
        result = dt_classify(test, tree)
        eval_post.append(evaluation(test[class_column], result, 'classification_score'))
        if len(eval_pre) != 0:
            print(f'pre-pruning: {eval_pre[index]}')
        print(f'post-pruning: {eval_post[index]}')
    if len(eval_pre) != 0:
        average_pre = sum(eval_pre)/len(eval_pre)
        print(f'average pre-pruning: {average_pre}')
    #determine the average of the performance metric over all folds
    average = sum(eval_post)/len(eval_post)
    print(f'average post-pruning: {average}')
    return [average, eval_post, eval_pre]

## Graph Tree
`graph_tree` graphs the decision tree and displays the PDF of the graph

In [35]:
def graph_tree(data, tree, name, index):
    g = graphviz.Digraph(name=name + str(index), directory='graph output')
    features = list(data)
    tree_graphic(features, tree, None, None, g)
    h = g.unflatten(stagger=2)
    h.view()

## Tree Graphic
`tree_graphic` uses graphviz to visualize the tree by connecting the nodes

In [36]:
def tree_graphic(features, tree, previous_node, previous_edge, g):
    for key, value in tree.items():
        p_edge = previous_edge
        if type(value) is dict:
            if key in features:
                g.node(str(previous_node) + '->' + tree[key]['label'].name + str(previous_edge), label=tree[key]['label'].name, height = "1", width = "1")
                p_node = str(previous_node) + '->' + tree[key]['label'].name + str(previous_edge)
            else:
                if list(tree[key])[0] == 'label':
                    g.node(previous_node + '->' + str(tree[key]['label'].output) + str(key), label= str(tree[key]['label'].output))
                    g.edge(previous_node, previous_node + '->' + str(tree[key]['label'].output) + str(key), label=str(key))
                else:
                    g.edge(previous_node, previous_node + '->' + tree[key][list(tree[key])[0]]['label'].name + str(key), label=str(key))
                p_edge = str(key)
                p_node = previous_node
            tree_graphic(features, value, p_node, p_edge, g)
    return None

## Squared Error
`squared_error` calculated the squared error of a sample of data relative to the mean of the data

In [37]:
def squared_error(data):
    error = 0
    predicted_response = data.mean()
    for sample in data:
        error += (sample - predicted_response)**2
    return error

## Best Feature Regression
`best_feature_regression` finds the best feature to use for the next node of a regression tree

In [38]:
def best_feature_regression(data, class_column):
    best = {}
    #find the median of each feature
    splits = data.median()
    #determine the squared error of each feature split
    for feature in data.keys():
        if feature != class_column:
            #create subset by splitting the data on the median of the feature
            subset1 = data[class_column][data[feature] <= splits[feature]]
            subset2 = data[class_column][data[feature] > splits[feature]]
            #if a subset is empty then no split possible
            if (len(subset1) == 0) or (len(subset2) == 0):
                continue
            #determine the MSE of the feature
            error = sum([squared_error(subset1), squared_error(subset2)])/len(data)
            best[feature] = error
    #if no splits possible return None
    if len(best) == 0:
        return None, None, None
    #find split that minimizes MSE
    error = best[min(best, key=best.get)]
    best = min(best, key=best.get)
    return best, round(splits[best], 5), round(error, 5)

## Desision Tree Regression
`decision_tree_regression` creates a decision tree for regression

In [39]:
def decision_tree_regression(_data, class_column):
    data = deepcopy(_data)
    #named tuple structure to be used as label for each node
    attr = namedtuple('attr', 'split name mse samples output')
    #find the MSE of the current node
    node_error = round(squared_error(data[class_column])/len(data), 5)
    #find the feature to use for the next node
    best_f, split, error = best_feature_regression(data, class_column)
    #if MSE is 0 or no more splits possible create leaf
    if node_error == 0 or best_f is None:
        node = {'label': attr(split = None, name = None, mse = node_error, samples = len(data), output = round(data[class_column].mean(), 5))}
        return node
    #create new dictionary entry as new node for the tree
    node = {best_f: {'True': {}, 'False': {}}}
    #add label to the node
    node[best_f]['label'] = attr(split = split, name = best_f + '<=' + str(split),
                                          mse = error, samples = len(data), output = round(data[class_column].mean(), 5))
    #create subsets for each branch of the new node
    subset1 = data[data[best_f] <= split]
    subset2 = data[data[best_f] > split]
    #find the next node or leaf
    if len(subset1) != 0:
        node[best_f]['True'] = decision_tree_regression(subset1, class_column)
    if len(subset2) != 0:
        node[best_f]['False'] = decision_tree_regression(subset2, class_column)
    return node

## Cross Validation Regression
`cross_validation_regression` K fold cross validation for a regression decision tree

In [40]:
def cross_validation_regression(data, class_column, post_prune = True):
    eval_pre = []
    eval_post = []
    #creates the validation and test/train sets
    folds, validation = k_fold_cross_validation_sets(data, 5, class_column, stratified=False)
    #run each fold as the test set
    for index, fold in enumerate(folds):
        #create train test sets from folds
        train, test = create_train_test(folds, index)
        #create decision tree
        tree = decision_tree_regression(train, class_column)
        #if pruning is selected then prune the tree
        if post_prune:
            #graph the un-pruned tree
            graph_tree(data, tree, data.name + ' pre', index)
            #determine performance of the un-pruned tree
            result = dt_classify(test, tree)
            eval_pre.append(evaluation(test[class_column], result, 'mse'))
            #determine number of paths in the tree
            path_count, length = num_paths(tree)
            new_path_count = 0
            #keep pruning until the number of paths in the tree stops changing
            while path_count > new_path_count:
                #find nodes to be pruned
                nodes = find_pruning_nodes(tree)
                #determine number of paths in the tree
                path_count, length = num_paths(tree)
                #test performance of pruning each node individually against the validation set
                for n in nodes:
                    #copy the tree
                    prune_tree = deepcopy(tree)
                    #determine performance of the tree
                    result = dt_classify(validation, prune_tree)
                    eval1 = evaluation(validation[class_column], result, 'mse')
                    #prune tree
                    prune_node(prune_tree, n)
                    #determine performance of pruned tree
                    result = dt_classify(validation, prune_tree)
                    eval2 = evaluation(validation[class_column], result, 'mse')
                    #if performance of pruned tree is not worse, then keep the node pruned
                    if eval2 >= eval1:
                        tree = prune_tree
                #count the new number of paths from the pruned tree
                new_path_count, length = num_paths(tree)
            #graph pruned tree
            graph_tree(data, tree, data.name + ' post', index)
        #if no pruning graph tree
        else:
            graph_tree(data, tree, data.name, index)
        #evaluate performance of the tree
        result = dt_classify(test, tree)
        eval_post.append(evaluation(test[class_column], result, 'mse'))
        if len(eval_pre) != 0:
            print(f'pre-pruning: {eval_pre[index]}')
        print(f'post-pruning: {eval_post[index]}')
    if len(eval_pre) != 0:
        average_pre = sum(eval_pre)/len(eval_pre)
        print(f'average pre-pruning: {average_pre}')
    #determine the average of the performance metric over all folds
    average = sum(eval_post)/len(eval_post)
    print(f'average post-pruning: {average}')
    return [average, eval_post]

In [41]:
cancer_data = load_breast_cancer('datasets/breast-cancer-wisconsin.data')
cancer_data.name = 'Breast Cancer'

In [42]:
cancer_data = cancer_data.astype(str)
cancer_data.name = 'Breast Cancer Cat'

In [43]:
%%time
cancer_cat_prune = cross_validation_classification(cancer_data, 'class', post_prune=True)

pre-pruning: 0.9324324324324325
post-pruning: 0.9459459459459459
pre-pruning: 0.8918918918918919
post-pruning: 0.918918918918919
pre-pruning: 0.8243243243243243
post-pruning: 0.8243243243243243
pre-pruning: 0.8918918918918919
post-pruning: 0.9459459459459459
pre-pruning: 0.8767123287671232
post-pruning: 0.863013698630137
average pre-pruning: 0.8834505738615327
average post-pruning: 0.8996297667530545
CPU times: total: 2.69 s
Wall time: 6.36 s


In [44]:
cancer_data = cancer_data.astype(float)
cancer_data.name = 'Breast Cancer Num'

In [45]:
%%time
cancer_num_prune = cross_validation_classification(cancer_data, 'class', post_prune=True)

pre-pruning: 0.9054054054054054
post-pruning: 0.8918918918918919
pre-pruning: 0.9054054054054054
post-pruning: 0.9054054054054054
pre-pruning: 0.9324324324324325
post-pruning: 0.918918918918919
pre-pruning: 0.9324324324324325
post-pruning: 0.8648648648648649
pre-pruning: 0.9452054794520548
post-pruning: 0.9452054794520548
average pre-pruning: 0.924176231025546
average post-pruning: 0.9052573121066272
CPU times: total: 2.23 s
Wall time: 3.87 s


In [46]:
car_data = load_car('datasets/car.data')
car_data.name = 'car'

In [47]:
%%time
car_class_prune = cross_validation_classification(car_data, 'class', post_prune=True)

pre-pruning: 0.9028776978417267
post-pruning: 0.8741007194244604
pre-pruning: 0.9424460431654677
post-pruning: 0.9028776978417267
pre-pruning: 0.927536231884058
post-pruning: 0.8985507246376812
pre-pruning: 0.9309090909090909
post-pruning: 0.8909090909090909
pre-pruning: 0.9236363636363636
post-pruning: 0.8945454545454545
average pre-pruning: 0.9254810854873414
average post-pruning: 0.8921967374716827
CPU times: total: 19.1 s
Wall time: 24.9 s


In [48]:
house_votes_data = load_house_votes('datasets/house-votes-84.data')
house_votes_data.name = 'house votes'

In [49]:
%%time
house_class_prune = cross_validation_classification(house_votes_data, 'class', post_prune=True)

pre-pruning: 0.9428571428571428
post-pruning: 0.9571428571428572
pre-pruning: 0.9714285714285714
post-pruning: 0.9571428571428572
pre-pruning: 0.8714285714285714
post-pruning: 0.9428571428571428
pre-pruning: 0.9714285714285714
post-pruning: 0.9714285714285714
pre-pruning: 0.9705882352941176
post-pruning: 0.9705882352941176
average pre-pruning: 0.945546218487395
average post-pruning: 0.9598319327731092
CPU times: total: 2.11 s
Wall time: 4.14 s


In [50]:
machine_data = load_machine('datasets/machine.data')
machine_data.name = 'machine'

In [51]:
%%time
machine_reg_prune = cross_validation_regression(machine_data, 'PRP', post_prune=True)

pre-pruning: 3469.600143099416
post-pruning: 30324.021428243876
pre-pruning: 11111.269736842105
post-pruning: 5736.723882531081
pre-pruning: 4472.077486023392
post-pruning: 20144.640507877037
pre-pruning: 3126.292398128655
post-pruning: 17428.324745911104
pre-pruning: 3777.925675675676
post-pruning: 27966.195248626496
average pre-pruning: 5191.433087953849
average post-pruning: 20319.981162637916
CPU times: total: 5.73 s
Wall time: 8.38 s


In [52]:
forest_data = load_forest_fires('datasets/forestfires.data')
forest_data.name = 'forest fires'

In [53]:
%%time
forest_reg_prune = cross_validation_regression(forest_data, 'area', post_prune=True)

pre-pruning: 753.4585705347523
post-pruning: 761.2104960424476
pre-pruning: 1728.4918212765956
post-pruning: 462.20523497092125
pre-pruning: 20317.112803693428
post-pruning: 20459.360724596754
pre-pruning: 38155.948650537626
post-pruning: 6222.235928356534
pre-pruning: 13998.573519957949
post-pruning: 5156.573904964261
average pre-pruning: 14990.71707320007
average post-pruning: 6612.317257786184
CPU times: total: 21.3 s
Wall time: 29.2 s


In [54]:
abalone_data = load_abalone('datasets/abalone.data')
abalone_data.name = 'abalone'

In [55]:
%%time
abalone_reg_prune = cross_validation_regression(abalone_data, 'rings', post_prune=True)

KeyboardInterrupt: 